<a href="https://colab.research.google.com/github/EddyGiusepe/Pytorch_to_Speech_to_Text/blob/main/Speech_Transcript_with_Hugging_Face_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h2 align="center">English Audio Speech-to-Text Transcript with Hugging Face</h2>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro


## [Wav2Vec2-Base-960h](https://huggingface.co/facebook/wav2vec2-base-960h)

Este modelo básico pré-treinado e ajustado em $960$ horas de ``Librispeech`` em áudio de fala amostrado de $16$kHz. Ao usar o modelo, certifique-se de que sua entrada de fala também seja amostrada em $16$Khz.


Eles mostram que aprender representações poderosas apenas a partir do áudio da fala, seguido pelo ajuste fino da fala transcrita, pode superar os melhores métodos **semi-supervisionados**, sendo conceitualmente mais simples. O [wav2vec 2.0](https://huggingface.co/facebook/wav2vec2-base-960h) mascara a entrada de fala no espaço latente e resolve uma **tarefa contrastiva*** definida sobre uma quantização das representações latentes que são aprendidas em conjunto. Experimentos usando todos os dados rotulados do ``Librispeech`` atingem $1,8/3,3$ **WER** nos conjuntos de teste limpos/outros. Ao reduzir a quantidade de dados rotulados para uma hora, o [wav2vec 2.0](https://huggingface.co/facebook/wav2vec2-base-960h) supera o estado da arte anterior no subconjunto de $100$ horas enquanto usa $100$ vezes menos dados rotulados. Usando apenas dez minutos de dados rotulados e pré-treinamento em $53$ mil horas de dados não-rotulados ainda alcança $4,8/8,2$ **WER**. Isso demonstra a viabilidade do reconhecimento de voz com quantidades limitadas de dados rotulados.

Para testar o modelo pré-treinado, [Hosted Inference API](https://huggingface.co/facebook/wav2vec2-base-960h), e assistir um Tutorial, pode seguir os seguintes links:

* [facebook/wav2vec2-base-960h](https://huggingface.co/facebook/wav2vec2-base-960h) and [Hosted Inference API](https://huggingface.co/facebook/wav2vec2-base-960h).

* [English Audio Speech-to-Text Transcript with Hugging Face](https://www.youtube.com/watch?v=dJAoK5zK36M)



In [1]:
# Instalamos transformers

! pip install -q transformers

     |████████████████████████████████| 4.0 MB 28.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 
     |████████████████████████████████| 895 kB 69.8 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 


In [2]:
# Intalamos as nossas bibliotecas

import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [3]:
# Aqui carregamos nosso modelo pré-treinado e tokenizer

tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:748: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Carregamos algum arquivo de áudio em Inglês

speech, rate = librosa.load("/content/drive/MyDrive/2_DEEP_LEARNING_REDES_NEURAIS_Jorge/2_SpeechBrain/speechbrain/samples/audio_samples/example1.wav",sr=16000)

In [5]:
import IPython.display as display

display.Audio("/content/drive/MyDrive/2_DEEP_LEARNING_REDES_NEURAIS_Jorge/2_SpeechBrain/speechbrain/samples/audio_samples/example1.wav", autoplay=True)

In [6]:
input_values = tokenizer(speech, return_tensors = 'pt').input_values

In [7]:
input_values

tensor([[0.0118, 0.0088, 0.0059,  ..., 0.0044, 0.0221, 0.0088]])

In [8]:
# Armazenar logits (previsões não normalizadas)

logits = model(input_values).logits

In [9]:
logits

tensor([[[ 12.0266, -27.1736, -26.9690,  ...,  -7.8541, -10.5646,  -8.2603],
         [ 11.8670, -26.5545, -26.3550,  ...,  -7.6593, -10.4945,  -8.1725],
         [ 11.9858, -25.9846, -25.8213,  ...,  -7.3364, -10.1871,  -8.0801],
         ...,
         [ 14.1830, -25.9861, -25.6892,  ...,  -7.2635,  -8.2811,  -6.5832],
         [ 14.4573, -24.5903, -24.3336,  ...,  -5.7868,  -7.0718,  -5.7210],
         [ 14.4571, -24.5904, -24.3337,  ...,  -5.7869,  -7.0719,  -5.7211]]],
       grad_fn=<AddBackward0>)

In [10]:
# Armazenar IDs previstos 

predicted_ids = torch.argmax(logits, dim =-1)

In [11]:
# decodificar o áudio para gerar texto

transcriptions = tokenizer.decode(predicted_ids[0])

In [12]:
# Finalmente visualizamos a transcrição de nosso áudio

print(transcriptions)

THE BIRCH CANOE SLID ON THE SMOOTH PLINKS
